In [1]:
import numpy as np
import pandas as pd
from scipy import linalg as la
import matplotlib.pyplot as plt

1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normali distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


In [72]:
N = 5000

x_1 = np.random.normal(loc = 0, scale = 1, size = N)
x_2 = x_1 + np.random.normal(loc = 0, scale = 3, size = N)
x_3 = (2 * x_1) + x_2

data = pd.DataFrame({'x1' : x_1, 'x2' : x_2, 'x3' : x_3})

covariance_matrix = np.cov(data.T)
print('Covariance matrix:\n', covariance_matrix)
l, V = la.eig(covariance_matrix)
print('\nEigenvalues:\n', l.real)
print('\nEigenvectors:\n', V)

U, spectrum, Vt = la.svd(data.T)
l_svd = (spectrum ** 2)/(N-1)
V_svd = U

if np.allclose(np.sort(l), np.sort(l_svd), atol = 0.05) == True and np.allclose(np.sort(V), np.sort(V_svd), atol = 0.05) == True:
    print("\nSVD yields to the same result")
else:
    print("\nSVD doesn't yield to the same result")
    
percentual_variabilities = l.real * 100 / np.sum(l.real)
print("\nPercent of dataset's variability:")

for v in percentual_variabilities[np.argsort(-percentual_variabilities)]:
    if v > 1e-3:
        print(f'{v:.3f}%')
    else:
        print(f'{v:.3e}%')

Covariance matrix:
 [[ 1.03781518  1.09713744  3.1727678 ]
 [ 1.09713744 10.11003021 12.30430508]
 [ 3.1727678  12.30430508 18.64984068]]

Eigenvalues:
 [2.77923969e+01 1.41026914e-15 2.00528921e+00]

Eigenvectors:
 [[-0.11968468 -0.81649658  0.56480874]
 [-0.57206192 -0.40824829 -0.71139194]
 [-0.81143129  0.40824829  0.41822554]]

SVD yields to the same result

Percent of dataset's variability:
93.270%
6.730%
4.733e-15%


We see one very high percent and the other two that are really small, one in particular is practically zero. This is due to the fact that there is only one indipendent variable, which is $x_1$, and the other two are just derived from $x_1$:

* $x_2$ is just $x_1$ plus another gaussian distribution with same mean and different standard deviation, so we expect low variability with respect to this variable
* $x_3$ is just a combination of $x_1$ and $x_2$ so its variability should be very very low, which is what we observe ($\cal o\left(10^{-15}\right)$)

In [77]:
i_min = np.argmin(l)

covariance_matrix_99 = np.copy(covariance_matrix)

for i in range(2):
    covariance_matrix_99 = np.delete(covariance_matrix_99, i_min, axis = i)
    
print('New covariance matrix:\n', covariance_matrix_99)
l, V = la.eig(covariance_matrix_99)
print('\nNew eigenvalues:\n', l.real)
print('\nNew eigenvectors:\n', V)

percentual_variabilities = l.real * 100 / np.sum(l.real)
print("\nPercent of dataset's variability:")

for v in percentual_variabilities[np.argsort(-percentual_variabilities)]:
    if v > 1e-3:
        print(f'{v:.3f}%')
    else:
        print(f'{v:.3e}%')

[[10.11003021 12.30430508]
 [12.30430508 18.64984068]]
New covariance matrix:
 [[10.11003021 12.30430508]
 [12.30430508 18.64984068]]

New eigenvalues:
 [ 1.35580345 27.40406744]

New eigenvectors:
 [[-0.8148146  -0.57972163]
 [ 0.57972163 -0.8148146 ]]

Percent of dataset's variability:
95.286%
4.714%


2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [ ]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 